# Azure AI Foundry Project Management

This notebook contains utilities to connect to your Azure AI Foundry project and enumerate its resources, including:

- Project properties
- Agents
- Vector stores 
- Tools
- Deployments
- Connections

The code uses the Azure AI Projects SDK to interact with your project.

In [17]:
import os
import json
from pprint import pprint
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.ai.agents import AgentsClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import ListSortOrder, FileSearchTool

# Load environment variables
load_dotenv()

# Get environment variables
PROJECT_ENDPOINT = os.getenv("AIPROJECT_ENDPOINT")
ASSISTANT_ID = os.getenv("ASSISTANT_ID")

# Check for required environment variables
if not PROJECT_ENDPOINT:
    print("⚠️ AIPROJECT_ENDPOINT environment variable is not set!")
else:
    print(f"✅ Project endpoint: {PROJECT_ENDPOINT}")

if not ASSISTANT_ID:
    print("⚠️ ASSISTANT_ID environment variable is not set!")
else:
    print(f"✅ Assistant ID: {ASSISTANT_ID}")

✅ Project endpoint: https://assistant-aura-alpha-resource.services.ai.azure.com/api/projects/assistant-aura-alpha
✅ Assistant ID: asst_hVh29mJp1DP5nzaVBIjqyNqe


## Connect to Azure AI Foundry Project

First, we'll create clients to connect to your Azure AI Foundry project. We'll create:
1. An `AIProjectClient` for general project operations
2. An `AgentsClient` for working with agents specifically

We'll use the DefaultAzureCredential for authentication, which tries multiple authentication methods.

In [4]:
# Create Azure credential
credential = DefaultAzureCredential()

# Initialize the project client
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=credential
)

# Initialize the agents client 
agents_client = AgentsClient(
    endpoint=PROJECT_ENDPOINT,
    credential=credential
)

print("✅ Clients initialized successfully")

✅ Clients initialized successfully


## Enumerate AI Agents

Let's list all the agents in your Azure AI Foundry project, including their properties such as:
- ID
- Name
- Instructions
- Tools and capabilities

In [5]:
def print_agent_details(agent):
    """Print detailed information about an agent"""
    print(f"\n{'=' * 50}")
    print(f"Agent: {agent.name}")
    print(f"ID: {agent.id}")
    print(f"Created: {agent.created_at}")
    print(f"Modified: {agent.modified_at}")
    print(f"Status: {agent.status}")
    print(f"Model: {agent.model}")
    
    if hasattr(agent, 'instructions') and agent.instructions:
        print("\nInstructions:")
        print(f"{agent.instructions[:500]}..." if len(agent.instructions) > 500 else agent.instructions)
    
    if hasattr(agent, 'tools') and agent.tools:
        print("\nTools:")
        for tool in agent.tools:
            print(f"- {tool.type}")
            if hasattr(tool, 'function'):
                print(f"  Function: {tool.function.name}")
                print(f"  Description: {tool.function.description}")
    print(f"{'=' * 50}\n")

# List all agents in the project
print("Listing all agents in the project...\n")

try:
    # List all agents
    agents_list = list(agents_client.list_agents())
    
    if not agents_list:
        print("No agents found in this project.")
    else:
        print(f"Found {len(agents_list)} agent(s):")
        for agent in agents_list:
            print_agent_details(agent)
    
    # Get our specific assistant if ID is available
    if ASSISTANT_ID:
        print("\nGetting details for the current assistant...\n")
        try:
            current_assistant = agents_client.get_agent(agent_id=ASSISTANT_ID)
            print_agent_details(current_assistant)
        except Exception as e:
            print(f"Error retrieving assistant with ID {ASSISTANT_ID}: {str(e)}")
            
except Exception as e:
    print(f"Error listing agents: {str(e)}")

Listing all agents in the project...

Found 1 agent(s):

Agent: aura-agent-alpha
ID: asst_hVh29mJp1DP5nzaVBIjqyNqe
Created: 2025-06-06 13:37:54+00:00
Error listing agents: 'Agent' object has no attribute 'modified_at'


## List Vector Stores

Vector stores are used to store embeddings for file search and retrieval. Let's list all the vector stores in your project.

## List Files

Let's enumerate all files that have been uploaded to the project.

In [25]:
try:
    # List all files
    files = list(agents_client.files.list())
    
    print(f"Found {len(files)} file(s):")
    for i, file in enumerate(files):
        print(f"\n{'=' * 50}")
        print(f"File {i+1}:")
        # print(f"ID: {file.id}")
        # print(f"Created: {file.created_at}")
        print(f"Filename: {file.capitalize}")
        # print(f"Purpose: {file.purpose}")
        # print(f"Size: {file.bytes} bytes")
        # print(f"Status: {file.status}")
        print(f"{'=' * 50}")
    
    if not files:
        print("No files found in this project.")
        
except Exception as e:
    print(f"Error listing files: {str(e)}")

Found 5 file(s):

File 1:
Filename: <built-in method capitalize of str object at 0x000001EF5A8C7FC0>

File 2:
Filename: <built-in method capitalize of str object at 0x000001EF5ABFB240>

File 3:
Filename: <built-in method capitalize of str object at 0x000001EF59601CF0>

File 4:
Filename: <built-in method capitalize of str object at 0x000001EF5C12B2B0>

File 5:
Filename: <built-in method capitalize of str object at 0x000001EF5ABF99B0>


Add the files to new vector store

In [15]:
# List all files
# List all files
files = list(agents_client.files.list())

# Extract file IDs from the file objects
file_ids = [f.id for f in files if hasattr(f, "id")]

# Create a new vector store with these files
vector_store = agents_client.vector_stores.create_and_poll(file_ids=file_ids, name="aura_vectorstore")

In [16]:
try:
    # List all vector stores
    vector_stores = list(agents_client.vector_stores.list())
    
    print(f"Found {len(vector_stores)} vector store(s):")
    for i, vs in enumerate(vector_stores):
        print(f"\n{'=' * 50}")
        print(f"Vector Store {i+1}:")
        print(f"ID: {vs.id}")
        print(f"Name: {vs.name}")
        print(f"Created: {vs.created_at}")
        print(f"Status: {vs.status}")
        
        # List files associated with this vector store
        print("\nAssociated Files:")
        try:
            files = list(agents_client.vector_store_files.list(vector_store_id=vs.id))
            if not files:
                print("  No files associated with this vector store.")
            else:
                for file in files:
                    print(f"  - File ID: {file.file_id}")
                    print(f"    Created: {file.created_at}")
                    print(f"    Status: {file.status}")
                    print()
        except Exception as file_err:
            print(f"  Error listing files for vector store {vs.id}: {str(file_err)}")
            
        print(f"{'=' * 50}")
    
    if not vector_stores:
        print("No vector stores found in this project.")
        
except Exception as e:
    print(f"Error listing vector stores: {str(e)}")

Found 1 vector store(s):

Vector Store 1:
ID: vs_AHkYSg1YwuI7hNAbn90EKMzd
Name: aura_vectorstore
Created: 2025-07-11 16:22:20+00:00
Status: VectorStoreStatus.COMPLETED

Associated Files:
  No files associated with this vector store.


# Create file search tool and add it to agent's tools

In [24]:
from pprint import pprint

# Create file search tool and agent

# Extract file IDs from the file objects
vectorstore_ids = [v.id for v in vector_stores if hasattr(v, "id")]

file_search = FileSearchTool(vector_store_ids=vectorstore_ids)

agent = agents_client.update_agent(
    agent_id=ASSISTANT_ID,
    tools=file_search.definitions,
    tool_resources=file_search.resources,
)

# dump properties of the agent

pprint(agent.__dict__ if hasattr(agent, "__dict__") else agent)


{'_data': {'created_at': 1749217074,
           'description': None,
           'id': 'asst_hVh29mJp1DP5nzaVBIjqyNqe',
           'instructions': 'You are a helpful assistant and can search '
                           'information from uploaded files',
           'metadata': {},
           'model': 'gpt-4o',
           'name': 'aura-agent-alpha',
           'object': 'assistant',
           'response_format': 'auto',
           'temperature': 1.0,
           'tool_resources': {'file_search': {'vector_store_ids': ['vs_AHkYSg1YwuI7hNAbn90EKMzd']}},
           'tools': [{'type': 'file_search'}],
           'top_p': 1.0}}


## List Model Deployments

Let's list all the model deployments available in your project. These are the models that can be used by your agents.

In [7]:
try:
    # List all deployments (models)
    deployments = list(project_client.deployments.list())
    
    print(f"Found {len(deployments)} deployment(s):")
    for i, deployment in enumerate(deployments):
        print(f"\n{'=' * 50}")
        print(f"Deployment {i+1}:")
        print(f"Name: {deployment.name}")
        if hasattr(deployment, 'model_name'):
            print(f"Model: {deployment.model_name}")
        if hasattr(deployment, 'model_version'):
            print(f"Version: {deployment.model_version}")
        if hasattr(deployment, 'model_publisher'):
            print(f"Publisher: {deployment.model_publisher}")
        if hasattr(deployment, 'status'):
            print(f"Status: {deployment.status}")
        print(f"{'=' * 50}")
    
    if not deployments:
        print("No deployments found in this project.")
        
except Exception as e:
    print(f"Error listing deployments: {str(e)}")

Found 2 deployment(s):

Deployment 1:
Name: gpt-4o-mini
Model: gpt-4o-mini
Version: 2024-07-18
Publisher: OpenAI

Deployment 2:
Name: gpt-4o
Model: gpt-4o
Version: 2024-11-20
Publisher: OpenAI


## List Connections

Let's list all the connections configured in your project. Connections allow your agents to interact with external services like Azure AI Search, Azure Blob Storage, etc.

In [6]:
try:
    # List all connections
    connections = list(project_client.connections.list())
    
    print(f"Found {len(connections)} connection(s):")
    for i, connection in enumerate(connections):
        print(f"\n{'=' * 50}")
        print(f"Connection {i+1}:")
        print(f"Name: {connection.name}")
        print(f"ID: {connection.id}")
        print(f"Type: {connection.type}")
        print(f"Created: {connection.created_at}")
        if hasattr(connection, 'properties'):
            print("\nProperties:")
            for key, value in connection.properties.items():
                # Don't print sensitive information
                if 'key' in key.lower() or 'secret' in key.lower() or 'password' in key.lower():
                    print(f"  {key}: [REDACTED]")
                else:
                    print(f"  {key}: {value}")
        print(f"{'=' * 50}")
    
    if not connections:
        print("No connections found in this project.")
        
except Exception as e:
    print(f"Error listing connections: {str(e)}")

Found 3 connection(s):

Connection 1:
Name: dacob-mbi0c18u-eastus2
ID: /subscriptions/b54ffc53-859b-4392-bce3-fbe32ca025bd/resourceGroups/ie-rg-middleware-dev/providers/Microsoft.CognitiveServices/accounts/assistant-aura-alpha-resource/projects/assistant-aura-alpha/connections/dacob-mbi0c18u-eastus2
Type: AIServices
Error listing connections: 'Connection' object has no attribute 'created_at'


## List and Manage Threads

Threads are conversation sessions with your agents. Let's list all the threads in your project and provide some utilities for managing them.

In [7]:
try:
    # List all threads
    threads = list(agents_client.threads.list())
    
    print(f"Found {len(threads)} thread(s):")
    for i, thread in enumerate(threads):
        print(f"\n{'=' * 50}")
        print(f"Thread {i+1}:")
        print(f"ID: {thread.id}")
        print(f"Created: {thread.created_at}")
        
        # Get messages in this thread
        try:
            messages = list(agents_client.messages.list(thread_id=thread.id))
            print(f"Messages: {len(messages)}")
            
            # Print the first few messages as a preview
            if messages:
                print("\nMessage Preview:")
                for j, msg in enumerate(messages[:3]):  # Show first 3 messages
                    print(f"  Message {j+1}:")
                    print(f"  Role: {msg.role}")
                    print(f"  Created: {msg.created_at}")
                    if hasattr(msg, 'text_messages') and msg.text_messages:
                        content = msg.text_messages[0].text.value
                        preview = content[:100] + "..." if len(content) > 100 else content
                        print(f"  Content: {preview}")
                    print()
                
                if len(messages) > 3:
                    print(f"  ... and {len(messages) - 3} more message(s)")
        except Exception as msg_err:
            print(f"Error listing messages for thread {thread.id}: {str(msg_err)}")
            
        print(f"{'=' * 50}")
    
    if not threads:
        print("No threads found in this project.")
        
except Exception as e:
    print(f"Error listing threads: {str(e)}")

# Function to delete a thread - uncomment and use when needed
"""
def delete_thread(thread_id):
    try:
        agents_client.threads.delete(thread_id=thread_id)
        print(f"Thread {thread_id} deleted successfully.")
    except Exception as e:
        print(f"Error deleting thread {thread_id}: {str(e)}")
        
# Example usage: delete_thread("your-thread-id")
"""

Found 55 thread(s):

Thread 1:
ID: thread_7p2zgxcXYyrjrOVen4kZTSuL
Created: 2025-07-10 15:50:54+00:00
Messages: 0

Thread 2:
ID: thread_YZrPbAmX4MaRtC4gTocTFXzz
Created: 2025-07-10 15:50:52+00:00
Messages: 0

Thread 3:
ID: thread_p8Bk5UmrEA4pMJYyFHLzNHYu
Created: 2025-07-10 15:50:52+00:00
Messages: 0

Thread 4:
ID: thread_QAI91PC4Vg2xubRyyMwak7G5
Created: 2025-07-10 15:50:50+00:00
Messages: 0

Thread 5:
ID: thread_WLpRl6sTo4VnXWXxIk5RPxKN
Created: 2025-07-10 15:50:50+00:00
Messages: 0

Thread 6:
ID: thread_Hiz0QQLv2HplMbRGVXtsioV9
Created: 2025-07-10 15:50:50+00:00
Messages: 0

Thread 7:
ID: thread_Oc9Xj0czgbIVLLc4FuchCknd
Created: 2025-07-10 15:50:50+00:00
Messages: 4

Message Preview:
  Message 1:
  Role: MessageRole.AGENT
  Created: 2025-07-10 15:51:08+00:00
  Content: Great! To recommend the right solution, let’s start with these questions based on your application:
...

  Message 2:
  Role: MessageRole.USER
  Created: 2025-07-10 15:51:06+00:00
  Content: What questions shall I ans

'\ndef delete_thread(thread_id):\n    try:\n        agents_client.threads.delete(thread_id=thread_id)\n        print(f"Thread {thread_id} deleted successfully.")\n    except Exception as e:\n        print(f"Error deleting thread {thread_id}: {str(e)}")\n\n# Example usage: delete_thread("your-thread-id")\n'

## Cleanup Resources

To properly clean up resources and close client connections, run the following cell when you're done.

In [ ]:
# Close client connections to release resources
if project_client:
    project_client.close()
    print("✅ Project client closed.")
    
if agents_client:
    agents_client.close()
    print("✅ Agents client closed.")

print("All resources cleaned up successfully.")